<a href="https://colab.research.google.com/github/saqiba123/Creating-ChatBot-using-python/blob/main/SayabiDevs_Challenge_1_ChatBot_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SayabiDevs Challenge 1: ChatBot in Python with respective JSON file


###Saqiba Juna
###Shayan Hassan
###Rayyan Minhaj

In [ ]:
#THIS IS OUR MAIN FILE, WHERE WE WILL RUN THE CHAT BOT
#PLEASE NOTE: THE JSON FILE HAS BEEN PURPOSELY LEFT UNCOMPLETE DEPENDING ON 
#NEEDS OF END-USER OR CLIENT, THE FILE CAN BE EDITED AND APPENDED ON DEMAND


import pandas as pd
#PANDAS NEEDED TO OPEN AND READ THE .CSV FILE
#csv file contains the different patterns and their respective tags 
#these tags are used to identify and mark the specific language to let
#the model know which response to output from the json file 

from sklearn.feature_extraction.text import TfidfVectorizer #needed to convert raw unfiltered data into TF-IDF form
#TF-IDF form helps quantify the importance of text in a language for machine learning

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential #used to create sequential deep learning models
from tensorflow.keras.layers import Dense #to generate a "dense" layer in the neural network
from tensorflow.keras.models import save_model #used to correlate variables to the respective models


training_data = pd.read_csv("training_data.csv")
training_data["patterns"] = training_data["patterns"].str.lower()

#helping vectorize the format in english language
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")

#now we will convert our data into an array format
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

le = LabelEncoder()

#converts the data into dataframe with tags column stored in its own variable
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})

#now through this dataframe we will manip the data into dummy/indicator variables
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()


#creating a DNN which follows the sequential order
chatbot = Sequential()


chatbot.add(Dense(30, input_shape=(len(training_data_tfidf[0]),)))

#this will set up the neural network by determinign number of dense layers in it
chatbot.add(Dense(8))
chatbot.add(Dense(8))
chatbot.add(Dense(6))

#we set multiple dense layers and then used the softmax acctivation func
chatbot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))

#fianlly chatbot.compile to set optimizer, show loss func values and accuracy to original result
chatbot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

#finally we fit the model and set num of epochs as 50
chatbot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=50, batch_size=32)
save_model(chatbot, "chatbot")


Epoch 1/50
1/1 [==============================] - 1s 660ms/step - loss: 0.8084 - accuracy: 0.3333
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 0.7675 - accuracy: 0.4167
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 0.7421 - accuracy: 0.4167
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 0.7225 - accuracy: 0.4167
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 0.7059 - accuracy: 0.4167
Epoch 6/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6911 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6775 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6648 - accuracy: 0.5000
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6526 - accuracy: 0.5000
Epoch 10/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6408 - accuracy: 0.5000
Epoch 11/50
1/1 [====================

In [ ]:
import pandas as pd #again used for importing data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder #helps classify the various features into numbers
import numpy as np #used for argmax func only
from tensorflow.keras.models import load_model #to load our previously created model above
import json #to load a json file 
import random #sort of a randomizer used to determine choice later on

training_data = pd.read_csv("training_data.csv")
chatbot = load_model("chatbot")
responses = json.load(open("responses.json", "r"))


#here we preprocess the inputs from training data using TF-IDF format
training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")
vectorizer.fit(training_data["patterns"])


#same thing as above
le = LabelEncoder()
le.fit(training_data["tags"])

#this function helps predict the response from input
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str.lower()]).toarray()
    predicted_proba = chatbot.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    return predicted_tag


#basic func that just continues taking in inputs (our questions) and giving output
#thru the above func we created (responses) 
def start_chat():
    print("---------------  AI Chat bot  ---------------")
    print("Ask any queries...")
    print("I will try to understand you and reply...")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        if inp == "EXIT":
            break
        else:
            if inp:
                tag = predict_tag(inp)
                response = random.choice(responses[tag])
                print("Response... : ", response)
            else:
                pass


start_chat()

---------------  AI Chat bot  ---------------
Ask any queries...
I will try to understand you and reply...
Type EXIT to quit...
Ask anything... : hello
Response... :  Hello! How can we help you?
Ask anything... : who are you
Response... :  Hi! SaybiDevs here! Ask me anything.
Ask anything... : EXIT
